# Context-FID Score Presentation
## Necessary packages and functions call

- Context-FID score: A useful metric measures how well the the synthetic time series windows ”fit” into the local context of the time series

In [4]:
import os
import torch
import numpy as np
import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))
from Utils.context_fid import Context_FID
from Utils.metric_utils import display_scores
from Utils.cross_correlation import CrossCorrelLoss

## Data Loading

Load original dataset and preprocess the loaded data.

In [5]:
iterations = 5
dataset_name = 'energy'
seq_length = 49
# ori_data = np.load('../toy_exp/samples/energy_ground_truth_32_train.npy')
ori_data = np.load(f'../energy_results/samples/{dataset_name}_norm_truth_{seq_length}_train.npy')  # Uncomment the line if dataset other than Sine is used.
fake_data = np.load('../energy_results/ddpm_fake_energy_0_to_1.npy')

## Context-FID Score

- The Frechet Inception distance-like score is based on unsupervised time series embeddings. It is able to score the fit of the fixed length synthetic samples into their context of (often much longer) true time series.

- The lowest scoring models correspond to the best performing models in downstream tasks

In [6]:
context_fid_score = []

for i in range(iterations):
    context_fid = Context_FID(ori_data[:], fake_data[:ori_data.shape[0]])
    context_fid_score.append(context_fid)
    print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')
      
display_scores(context_fid_score)

Iter 0:  context-fid = 0.11474754635038545 

Iter 1:  context-fid = 0.11272083864659964 

Iter 2:  context-fid = 0.13051291753401065 

Iter 3:  context-fid = 0.12563252713605402 

Iter 4:  context-fid = 0.12218864509883079 

Final Score:  0.12116049495317609 ± 0.00922677670332609


## Correlational Score

- The metric uses the absolute error of the auto-correlation estimator by real data and synthetic data as the metric to assess the temporal dependency.

- For d > 1, it uses the l1-norm of the difference between cross correlation matrices.

In [7]:
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

In [8]:
x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)
min_len = min(x_real.shape[0], x_fake.shape[0])
x_real = x_real[:min_len]
x_fake = x_fake[:min_len]
assert x_real.shape[2] == x_fake.shape[2], "Real and fake data must have the same shape except for the batch dimension."
correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

Iter 0:  cross-correlation = 4.389076102171767 

Iter 1:  cross-correlation = 3.914363407322783 

Iter 2:  cross-correlation = 5.643304498817473 

Iter 3:  cross-correlation = 6.9368043824817365 

Iter 4:  cross-correlation = 5.537986145641662 

Final Score:  5.2843069072870845 ± 1.4694404533156824


In [9]:
import os
import pandas as pd
import numpy as np
import scipy.stats

def compute_score(results, confidence=0.95):
    """Return mean ± CI string like display_scores."""
    mean = np.mean(results)
    sigma = scipy.stats.sem(results)
    sigma = sigma * scipy.stats.t.ppf((1 + confidence) / 2., len(results)-1)
    return f"{mean:.4f} ± {sigma:.4f}"

# Example usage after your iterations
context_fid_result = compute_score(context_fid_score)
corr_result = compute_score(correlational_score)

results = {
    "Metric": ["Context-FID Score", "Correlation Score"],
    "Result": [context_fid_result, corr_result]
}

results_df = pd.DataFrame(results)

save_folder = "../figures/"
os.makedirs(save_folder, exist_ok=True)

save_path = os.path.join(save_folder, f"{dataset_name}_metrics2.csv")
results_df.to_csv(save_path, index=False)

print(f"✅ Compact metrics saved to {save_path}")

✅ Compact metrics saved to ../figures/energy_metrics2.csv
